In [37]:
import tensorflow
import cv2
import os
import random
from tqdm import tqdm

In [53]:
training_path = os.path.join('dataset', 'test')
training_files_true = [os.path.join(training_path, 'true', x) for x in os.listdir(os.path.join(training_path, 'true'))]
training_files_false = [os.path.join(training_path, 'false', x) for x in
                        os.listdir(os.path.join(training_path, 'false'))]

dataset = training_files_true + training_files_false
random.shuffle(dataset)

In [54]:

TP = TN = FP = FN = 0
for img_path in tqdm(dataset):
    img = cv2.imread(img_path)
    etiquette = img_path.split('\\')[2] == 'true'
    numbers_bigger_than = (img[img > 220]).sum()
    prediction = numbers_bigger_than > 2000
    if prediction and etiquette:
        TP += 1
    elif prediction and not etiquette:
        FP += 1
    elif not prediction and etiquette:
        FN += 1
    else:
        TN += 1

100%|██████████| 19630/19630 [01:35<00:00, 206.63it/s]


In [55]:
print(TP, TN, FN, FP)

10032 7048 308 2242


In [56]:
accuracy = (TP + TN) / (TP + FN + TN + FP)
recall = TP / (TP + FN)
precision = TP / (TP + FP)
print(accuracy, recall, precision)

0.870096790626592 0.9702127659574468 0.8173374613003096
